# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Tidy up callback display
    - Not every iteration
    - Show meaningful metric like objective score
- Optimise for single wind speed for better initial estimate
    - Power not LCOE
    - Different values for below/at rated power
- Increase fidelity
    - Assess difference in speed of calculation and optimal values
- Loop over wind direction
    - Parallelise using PQDM
- Assess results
    - Sensitivity to wind direction/speed
    - Variation in sensitivity between turbines
    - Increase number of turbines

## Setup

In [1]:
# import libraries
import logging

import numpy as np
from scipy import optimize

import utils

In [2]:
# define constants
yaw_scale = 30

## Baseline values

In [3]:
# run baseline simulation
sim_res_base = utils.run_sim()

In [4]:
# ensure probabilities (wind direction and speed) total 1
Sector_frequency = sim_res_base.Sector_frequency
if not np.isclose(Sector_frequency.sum(), 1):
    logging.warning(
        f"Sector frequency renormalised as total probability was {Sector_frequency.sum().values}"
    )
    Sector_frequency = Sector_frequency / Sector_frequency.sum()
P = sim_res_base.P
if not np.isclose(P.sum(), 1):
    logging.warning(f"P renormalised as total probability was {P.sum().values}")
    P = P / P.sum()

In [5]:
# calculate baseline metrics
aep_base, lcoe_base, _ = utils.calc_metrics(
    sim_res=sim_res_base,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)
lcoe_direction_base, _, lcoe_overall_base, _ = utils.aggregate_metrics(
    aep=aep_base, lcoe=lcoe_base, Sector_frequency=Sector_frequency
)

AEP [GWh]: 79.174
LCoE [USD/MWh]: 44.477
Capacity factor [%]: 50.177


## Optimise across for a single wind direction

In [6]:
# define constants
wd = 270
yaw_shape = (len(sim_res_base.wt), 1, len(sim_res_base.ws))

In [7]:
# define objective function
def obj_single(yaw_norm):
    sim_res = utils.run_sim(yaw=yaw_norm.reshape(yaw_shape) * yaw_scale, wd=wd)
    aep, lcoe, _ = utils.calc_metrics(
        sim_res=sim_res,
        sim_res_base=sim_res_base,
        Sector_frequency=Sector_frequency,
        P=P,
    )
    _, _, lcoe_overall, _ = utils.aggregate_metrics(
        aep=aep, lcoe=lcoe, Sector_frequency=Sector_frequency
    )
    obj = (lcoe_overall / lcoe_direction_base.sel(wd=wd)).values.tolist()
    return obj

In [8]:
# check objective function correct for baseline case
assert np.isclose(obj_single(np.zeros(yaw_shape)), 1)

In [9]:
# run optimisation with iteration counter display
n = 0


def _callback(x):
    global n
    print(f"--- Iteration {n} ---")
    n += 1


res = optimize.minimize(
    fun=obj_single, x0=np.zeros(yaw_shape).ravel() + 1 / yaw_scale, callback=_callback
)
res

--- Iteration 0 ---
--- Iteration 1 ---
--- Iteration 2 ---
--- Iteration 3 ---
--- Iteration 4 ---
--- Iteration 5 ---
--- Iteration 6 ---
--- Iteration 7 ---
--- Iteration 8 ---
--- Iteration 9 ---
--- Iteration 10 ---
--- Iteration 11 ---
--- Iteration 12 ---
--- Iteration 13 ---
--- Iteration 14 ---
--- Iteration 15 ---
--- Iteration 16 ---
--- Iteration 17 ---
--- Iteration 18 ---
--- Iteration 19 ---
--- Iteration 20 ---
--- Iteration 21 ---
--- Iteration 22 ---
--- Iteration 23 ---
--- Iteration 24 ---
--- Iteration 25 ---
--- Iteration 26 ---
--- Iteration 27 ---
--- Iteration 28 ---
--- Iteration 29 ---
--- Iteration 30 ---
--- Iteration 31 ---
--- Iteration 32 ---
--- Iteration 33 ---
--- Iteration 34 ---
--- Iteration 35 ---
--- Iteration 36 ---
--- Iteration 37 ---
--- Iteration 38 ---
--- Iteration 39 ---
--- Iteration 40 ---
--- Iteration 41 ---
--- Iteration 42 ---
--- Iteration 43 ---
--- Iteration 44 ---
--- Iteration 45 ---
--- Iteration 46 ---
--- Iteration 47 ---
--

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.9921584645055506
        x: [ 3.333e-02  3.333e-02 ...  3.333e-02  3.333e-02]
      nit: 220
      jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
            ...
            [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
            [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
     nfev: 66080
     njev: 236